In [25]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
import seaborn as sns

In [26]:
transactions = pd.read_csv('../data/twm_transactions.csv', delimiter=';')
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77580 entries, 0 to 77579
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tran_id        77580 non-null  int64  
 1   acct_nbr       77580 non-null  int64  
 2   tran_amt       77580 non-null  float64
 3   principal_amt  77580 non-null  float64
 4   interest_amt   77580 non-null  float64
 5   new_balance    77580 non-null  float64
 6   tran_date      77580 non-null  object 
 7   tran_time      77580 non-null  int64  
 8   channel        77580 non-null  object 
 9   tran_code      77580 non-null  object 
dtypes: float64(4), int64(3), object(3)
memory usage: 5.9+ MB


In [27]:
# id should be dropped
transactions.tran_id.nunique()

155

In [28]:
#tran_code should be hot_encoded
transactions.tran_code.nunique()

13

In [29]:
#tran_channel should be hot_encoded
transactions.channel.nunique()

11

In [30]:
# let's drop date and time, not important, we should also group by acct

# Cleaning

In [31]:
transactions = transactions.drop(['tran_id','tran_date','tran_time'], axis=1)
transactions

,acct_nbr,tran_amt,principal_amt,interest_amt,new_balance,channel,tran_code
0,13625623,0.00,0.00,0.0,3753.34,A,IQ
1,13628392,0.00,0.00,0.0,254.49,V,IQ
2,13630842,-97.57,-97.57,0.0,3819.56,P,WD
3,13631412,-0.15,-0.15,0.0,224.05,,FK
4,13625722,0.00,0.00,0.0,240.55,B,IQ
...,...,...,...,...,...,...,...
77575,13631752,-95.71,-95.71,0.0,95.71,A,TR
77576,13626772,-93.90,-93.90,0.0,824.36,P,WD
77577,13625262,-10.35,-10.35,0.0,10.36,P,WD
77578,13627052,-423.80,-423.80,0.0,162.28,E,WD


In [32]:
# group by acct_nbr
acct_transactions = transactions.groupby('acct_nbr').sum().drop('new_balance', axis=1)
acct_transactions

,tran_amt,principal_amt,interest_amt
acct_nbr,,,
13624802,-244.99,-244.99,0.00
13624803,482.32,465.30,17.02
13624842,-2518.14,-2519.22,1.08
13624843,1929.66,1912.47,17.19
13624852,-164.29,-164.29,0.00
...,...,...,...
4561143213634830,25.78,-53.74,79.61
4561143213634840,471.14,458.93,12.21
4561143213634850,3202.51,3161.24,41.27


In [33]:
# add transaction counts
acct_transactions['tran_count'] = transactions.groupby('acct_nbr').count()['tran_amt']
acct_transactions

,tran_amt,principal_amt,interest_amt,tran_count
acct_nbr,,,,
13624802,-244.99,-244.99,0.00,99
13624803,482.32,465.30,17.02,26
13624842,-2518.14,-2519.22,1.08,99
13624843,1929.66,1912.47,17.19,33
13624852,-164.29,-164.29,0.00,99
...,...,...,...,...
4561143213634830,25.78,-53.74,79.61,77
4561143213634840,471.14,458.93,12.21,39
4561143213634850,3202.51,3161.24,41.27,23


# Scaling

In [34]:
# Scaling
from sklearn.preprocessing import MinMaxScaler

num_feats = acct_transactions.dtypes[(acct_transactions.dtypes != 'object') & (acct_transactions.columns != 'marital_status')].index.tolist()
scaler = MinMaxScaler()
scaled_transactions = pd.DataFrame(scaler.fit_transform(acct_transactions[num_feats].astype(float)))

In [35]:
# add columns back

scaled_transactions.columns = num_feats
scaled_transactions

,tran_amt,principal_amt,interest_amt,tran_count
0,0.600630,0.607446,0.000000,0.636364
1,0.626279,0.632778,0.016386,0.162338
2,0.520468,0.526336,0.001040,0.636364
3,0.677318,0.684391,0.016550,0.207792
4,0.603476,0.610324,0.000000,0.636364
...,...,...,...,...
1393,0.610179,0.614267,0.076645,0.493506
1394,0.625884,0.632551,0.011755,0.246753
1395,0.722205,0.728928,0.039733,0.142857
1396,0.609270,0.616183,0.000000,0.240260


In [36]:
# export

scaled_transactions.to_csv('../data/acct_transactions.csv')